In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
import re

In [2]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

  # Initialise & Explore Data
 

In [3]:
movies_df= pd.read_csv(r"C:\git-learn\cleaning_data\movies.csv\movies.csv")
movies_df.head(5)

,MOVIES,YEAR,GENRE,RATING,ONE-LINE,STARS,VOTES,RunTime,Gross
0,Blood Red Sky,(2021),"\nAction, Horror, Thriller",6.1,\nA woman with a mysterious illness is forced ...,\n Director:\nPeter Thorwarth\n| \n Star...,"21,062",121.0,NaN
1,Masters of the Universe: Revelation,(2021– ),"\nAnimation, Action, Adventure",5.0,\nThe war for Eternia begins again in what may...,"\n \n Stars:\nChris Wood, \nSara...","17,870",25.0,NaN
2,The Walking Dead,(2010–2022),"\nDrama, Horror, Thriller",8.2,\nSheriff Deputy Rick Grimes wakes up from a c...,"\n \n Stars:\nAndrew Lincoln, \n...","885,805",44.0,NaN
3,Rick and Morty,(2013– ),"\nAnimation, Adventure, Comedy",9.2,\nAn animated series that follows the exploits...,"\n \n Stars:\nJustin Roiland, \n...","414,849",23.0,NaN
4,Army of Thieves,(2021),"\nAction, Crime, Horror",NaN,"\nA prequel, set before the events of Army of ...",\n Director:\nMatthias Schweighöfer\n| \n ...,NaN,NaN,NaN


In [4]:
movies_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9999 entries, 0 to 9998
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   MOVIES    9999 non-null   object 
 1   YEAR      9355 non-null   object 
 2   GENRE     9919 non-null   object 
 3   RATING    8179 non-null   float64
 4   ONE-LINE  9999 non-null   object 
 5   STARS     9999 non-null   object 
 6   VOTES     8179 non-null   object 
 7   RunTime   7041 non-null   float64
 8   Gross     460 non-null    object 
dtypes: float64(2), object(7)
memory usage: 703.2+ KB


In [5]:
movies_df.shape

(9999, 9)

In [6]:
movies_columns = movies_df.columns
movies_columns

Index(['MOVIES', 'YEAR', 'GENRE', 'RATING', 'ONE-LINE', 'STARS', 'VOTES',
       'RunTime', 'Gross'],
      dtype='object')

In [7]:
movies_df.isnull().sum()

MOVIES         0
YEAR         644
GENRE         80
RATING      1820
ONE-LINE       0
STARS          0
VOTES       1820
RunTime     2958
Gross       9539
dtype: int64

In [8]:
movies_df.describe()

,RATING,RunTime
count,8179.000000,7041.000000
mean,6.921176,68.688539
std,1.220232,47.258056
min,1.100000,1.000000
25%,6.200000,36.000000
50%,7.100000,60.000000
75%,7.800000,95.000000
max,9.900000,853.000000


In [9]:
movies_df.isnull().sum()/ movies_df.shape[0]

MOVIES      0.000000
YEAR        0.064406
GENRE       0.008001
RATING      0.182018
ONE-LINE    0.000000
STARS       0.000000
VOTES       0.182018
RunTime     0.295830
Gross       0.953995
dtype: float64

In [10]:
movies_df.duplicated().sum()

431

# DATA Cleaning

What I'd done in this section are removinf duplicates, imputation, removing white space, and correcting Dtypes

In [11]:
movies_df.head(10)

,MOVIES,YEAR,GENRE,RATING,ONE-LINE,STARS,VOTES,RunTime,Gross
0,Blood Red Sky,(2021),"\nAction, Horror, Thriller",6.1,\nA woman with a mysterious illness is forced ...,\n Director:\nPeter Thorwarth\n| \n Star...,"21,062",121.0,NaN
1,Masters of the Universe: Revelation,(2021– ),"\nAnimation, Action, Adventure",5.0,\nThe war for Eternia begins again in what may...,"\n \n Stars:\nChris Wood, \nSara...","17,870",25.0,NaN
2,The Walking Dead,(2010–2022),"\nDrama, Horror, Thriller",8.2,\nSheriff Deputy Rick Grimes wakes up from a c...,"\n \n Stars:\nAndrew Lincoln, \n...","885,805",44.0,NaN
3,Rick and Morty,(2013– ),"\nAnimation, Adventure, Comedy",9.2,\nAn animated series that follows the exploits...,"\n \n Stars:\nJustin Roiland, \n...","414,849",23.0,NaN
4,Army of Thieves,(2021),"\nAction, Crime, Horror",NaN,"\nA prequel, set before the events of Army of ...",\n Director:\nMatthias Schweighöfer\n| \n ...,NaN,NaN,NaN
5,Outer Banks,(2020– ),"\nAction, Crime, Drama",7.6,\nA group of teenagers from the wrong side of ...,"\n \n Stars:\nChase Stokes, \nMa...","25,858",50.0,NaN
6,The Last Letter from Your Lover,(2021),"\nDrama, Romance",6.8,\nA pair of interwoven stories set in the past...,\n Director:\nAugustine Frizzell\n| \n S...,"5,283",110.0,NaN
7,Dexter,(2006–2013),"\nCrime, Drama, Mystery",8.6,"\nBy day, mild-mannered Dexter is a blood-spat...","\n \n Stars:\nMichael C. Hall, \...","665,387",53.0,NaN
8,Never Have I Ever,(2020– ),\nComedy,7.9,\nThe complicated life of a modern-day first g...,\n \n Stars:\nMaitreyi Ramakrish...,"34,530",30.0,NaN
9,Virgin River,(2019– ),"\nDrama, Romance",7.4,"\nSeeking a fresh start, nurse practitioner Me...",\n \n Stars:\nAlexandra Breckenr...,"27,279",44.0,NaN


In [12]:
#REMOVING DUPLICATES
movies_df = movies_df.drop_duplicates()

In [13]:
# Extract Start and End Year
movies_df[['START_YEAR', 'END_YEAR']] = movies_df['YEAR'].str.extract(r'(\d{4})[-–]?\s*(\d{4})?')

In [14]:
# Convert to numeric (to handle NaN)
movies_df['START_YEAR'] = pd.to_numeric(movies_df['START_YEAR'], errors='coerce')
movies_df['END_YEAR'] = pd.to_numeric(movies_df['END_YEAR'], errors='coerce')

In [15]:
# **Fix: Use .fillna('') before str.contains() to avoid NaN issues**
single_year_mask = movies_df['YEAR'].str.match(r'^\(\d{4}\)$', na=False)

# If YEAR is a single year (e.g., "(2021)"), copy START_YEAR to END_YEAR
movies_df.loc[single_year_mask, 'END_YEAR'] = movies_df['START_YEAR']

In [16]:
# Handle NaN cases:
# - If 'YEAR' is NaN, set both START_YEAR and END_YEAR to "No Specified Date"
# - If 'END_YEAR' is NaN (e.g., '2021– ' meaning ongoing series), set it to "Ongoing"
movies_df[['START_YEAR', 'END_YEAR']] = movies_df[['START_YEAR', 'END_YEAR']].fillna({'START_YEAR': 'No Specified Date', 'END_YEAR': 'No Specified Date'})
movies_df.loc[movies_df['END_YEAR'] == 'nan', 'END_YEAR'] = 'Ongoing'  # Fix for ongoing series


In [17]:
movies_df.head(5)

,MOVIES,YEAR,GENRE,RATING,ONE-LINE,STARS,VOTES,RunTime,Gross,START_YEAR,END_YEAR
0,Blood Red Sky,(2021),"\nAction, Horror, Thriller",6.1,\nA woman with a mysterious illness is forced ...,\n Director:\nPeter Thorwarth\n| \n Star...,"21,062",121.0,NaN,2021.0,2021.0
1,Masters of the Universe: Revelation,(2021– ),"\nAnimation, Action, Adventure",5.0,\nThe war for Eternia begins again in what may...,"\n \n Stars:\nChris Wood, \nSara...","17,870",25.0,NaN,2021.0,No Specified Date
2,The Walking Dead,(2010–2022),"\nDrama, Horror, Thriller",8.2,\nSheriff Deputy Rick Grimes wakes up from a c...,"\n \n Stars:\nAndrew Lincoln, \n...","885,805",44.0,NaN,2010.0,2022.0
3,Rick and Morty,(2013– ),"\nAnimation, Adventure, Comedy",9.2,\nAn animated series that follows the exploits...,"\n \n Stars:\nJustin Roiland, \n...","414,849",23.0,NaN,2013.0,No Specified Date
4,Army of Thieves,(2021),"\nAction, Crime, Horror",NaN,"\nA prequel, set before the events of Army of ...",\n Director:\nMatthias Schweighöfer\n| \n ...,NaN,NaN,NaN,2021.0,2021.0


In [18]:
#fillna
movies_df.loc[movies_df['YEAR'].str.contains('–', na=False) & movies_df['END_YEAR'].isna(), 'END_YEAR'] = "Ongoing"


In [19]:
movies_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9568 entries, 0 to 9998
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   MOVIES      9568 non-null   object 
 1   YEAR        9026 non-null   object 
 2   GENRE       9490 non-null   object 
 3   RATING      8168 non-null   float64
 4   ONE-LINE    9568 non-null   object 
 5   STARS       9568 non-null   object 
 6   VOTES       8168 non-null   object 
 7   RunTime     7008 non-null   float64
 8   Gross       460 non-null    object 
 9   START_YEAR  9568 non-null   object 
 10  END_YEAR    9568 non-null   object 
dtypes: float64(2), object(9)
memory usage: 1.1+ MB


In [20]:
#Removing extra space in MOVIES column
movies_df['MOVIES'] = movies_df['MOVIES'].str.strip()

In [21]:
movies_df['GENRE'] = movies_df['GENRE'].str.replace('\n', '', regex=False)
movies_df['GENRE'] = movies_df['GENRE'].str.strip()

In [22]:
movies_df.RATING = movies_df.RATING.fillna(value = 0)

In [23]:
# Remove commas (if numbers are formatted as strings)
movies_df['VOTES'] = movies_df['VOTES'].astype(str).str.replace(',', '')

# Convert to numeric, forcing errors to NaN
movies_df['VOTES'] = pd.to_numeric(movies_df['VOTES'], errors='coerce')

# Fill NaN values with 0
movies_df['VOTES'] = movies_df['VOTES'].fillna(0)


In [24]:
#Runtime to mean
movies_df.RunTime = movies_df.RunTime.fillna(movies_df['RunTime'].mean()).round(2) 

In [25]:
movies_df.head(10)

,MOVIES,YEAR,GENRE,RATING,ONE-LINE,STARS,VOTES,RunTime,Gross,START_YEAR,END_YEAR
0,Blood Red Sky,(2021),"Action, Horror, Thriller",6.1,\nA woman with a mysterious illness is forced ...,\n Director:\nPeter Thorwarth\n| \n Star...,21062.0,121.00,NaN,2021.0,2021.0
1,Masters of the Universe: Revelation,(2021– ),"Animation, Action, Adventure",5.0,\nThe war for Eternia begins again in what may...,"\n \n Stars:\nChris Wood, \nSara...",17870.0,25.00,NaN,2021.0,No Specified Date
2,The Walking Dead,(2010–2022),"Drama, Horror, Thriller",8.2,\nSheriff Deputy Rick Grimes wakes up from a c...,"\n \n Stars:\nAndrew Lincoln, \n...",885805.0,44.00,NaN,2010.0,2022.0
3,Rick and Morty,(2013– ),"Animation, Adventure, Comedy",9.2,\nAn animated series that follows the exploits...,"\n \n Stars:\nJustin Roiland, \n...",414849.0,23.00,NaN,2013.0,No Specified Date
4,Army of Thieves,(2021),"Action, Crime, Horror",0.0,"\nA prequel, set before the events of Army of ...",\n Director:\nMatthias Schweighöfer\n| \n ...,0.0,68.87,NaN,2021.0,2021.0
5,Outer Banks,(2020– ),"Action, Crime, Drama",7.6,\nA group of teenagers from the wrong side of ...,"\n \n Stars:\nChase Stokes, \nMa...",25858.0,50.00,NaN,2020.0,No Specified Date
6,The Last Letter from Your Lover,(2021),"Drama, Romance",6.8,\nA pair of interwoven stories set in the past...,\n Director:\nAugustine Frizzell\n| \n S...,5283.0,110.00,NaN,2021.0,2021.0
7,Dexter,(2006–2013),"Crime, Drama, Mystery",8.6,"\nBy day, mild-mannered Dexter is a blood-spat...","\n \n Stars:\nMichael C. Hall, \...",665387.0,53.00,NaN,2006.0,2013.0
8,Never Have I Ever,(2020– ),Comedy,7.9,\nThe complicated life of a modern-day first g...,\n \n Stars:\nMaitreyi Ramakrish...,34530.0,30.00,NaN,2020.0,No Specified Date
9,Virgin River,(2019– ),"Drama, Romance",7.4,"\nSeeking a fresh start, nurse practitioner Me...",\n \n Stars:\nAlexandra Breckenr...,27279.0,44.00,NaN,2019.0,No Specified Date


In [26]:
# Remove "Add a Plot"
movies_df['ONE-LINE'] = movies_df['ONE-LINE'].str.replace(r'\nAdd a Plot\n', '', regex=True) 

# Replace newlines with spaces
movies_df['ONE-LINE'] = movies_df['ONE-LINE'].str.replace(r'\n', ' ', regex=True)  

# Remove special characters except ., and spaces
movies_df['ONE-LINE'] = movies_df['ONE-LINE'].str.replace(r'[^\w\s.,]', '', regex=True)  

# Remove leading/trailing spaces
movies_df['ONE-LINE'] = movies_df['ONE-LINE'].str.strip() 

In [27]:
# Function to extract director
def extract_director(text):
    match = re.search(r"Director:\s*([^\n|]+)", text)
    return match.group(1).strip() if match else None #remove extra space

In [28]:
# Function to extract actors
def extract_actors(text):
    match = re.search(r"Stars:\s*([\s\S]+)", text)  # Capture everything after "Stars:"
    if match:
        actors = match.group(1).strip().split(",")  # Split by commas
        return [actor.strip() for actor in actors]  # Remove extra spaces
    return None

In [29]:
# Apply functions to extract data
movies_df['DIRECTOR'] = movies_df['STARS'].apply(extract_director)
movies_df['ACTORS'] = movies_df['STARS'].apply(extract_actors)


In [30]:
movies_df['ACTORS'] = movies_df['ACTORS'].apply(lambda x: ', '.join(x) if isinstance(x, list) else x) # removing Bracket

In [31]:
# Group the DataFrame by 'ONE-LINE' and fill NaN values in 'DIRECTOR' and 'ACTOR' by the most frequent value in each group
movies_df['DIRECTOR'] = movies_df.groupby('ONE-LINE')['DIRECTOR'].transform(lambda x: x.fillna(x.mode()[0] if not x.mode().empty else 'Unknown'))
movies_df['ACTORS'] = movies_df.groupby('ONE-LINE')['ACTORS'].transform(lambda x: x.fillna(x.mode()[0] if not x.mode().empty else 'Unknown'))

In [32]:
movies_df.head(5)

,MOVIES,YEAR,GENRE,RATING,ONE-LINE,STARS,VOTES,RunTime,Gross,START_YEAR,END_YEAR,DIRECTOR,ACTORS
0,Blood Red Sky,(2021),"Action, Horror, Thriller",6.1,A woman with a mysterious illness is forced in...,\n Director:\nPeter Thorwarth\n| \n Star...,21062.0,121.00,NaN,2021.0,2021.0,Peter Thorwarth,"Peri Baumeister, Carl Anton Koch, Alexander Sc..."
1,Masters of the Universe: Revelation,(2021– ),"Animation, Action, Adventure",5.0,The war for Eternia begins again in what may b...,"\n \n Stars:\nChris Wood, \nSara...",17870.0,25.00,NaN,2021.0,No Specified Date,Unknown,"Chris Wood, Sarah Michelle Gellar, Lena Headey..."
2,The Walking Dead,(2010–2022),"Drama, Horror, Thriller",8.2,Sheriff Deputy Rick Grimes wakes up from a com...,"\n \n Stars:\nAndrew Lincoln, \n...",885805.0,44.00,NaN,2010.0,2022.0,Unknown,"Andrew Lincoln, Norman Reedus, Melissa McBride..."
3,Rick and Morty,(2013– ),"Animation, Adventure, Comedy",9.2,An animated series that follows the exploits o...,"\n \n Stars:\nJustin Roiland, \n...",414849.0,23.00,NaN,2013.0,No Specified Date,Unknown,"Justin Roiland, Chris Parnell, Spencer Grammer..."
4,Army of Thieves,(2021),"Action, Crime, Horror",0.0,"A prequel, set before the events of Army of th...",\n Director:\nMatthias Schweighöfer\n| \n ...,0.0,68.87,NaN,2021.0,2021.0,Matthias Schweighöfer,"Matthias Schweighöfer, Nathalie Emmanuel, Ruby..."


In [33]:
movies_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9568 entries, 0 to 9998
Data columns (total 13 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   MOVIES      9568 non-null   object 
 1   YEAR        9026 non-null   object 
 2   GENRE       9490 non-null   object 
 3   RATING      9568 non-null   float64
 4   ONE-LINE    9568 non-null   object 
 5   STARS       9568 non-null   object 
 6   VOTES       9568 non-null   float64
 7   RunTime     9568 non-null   float64
 8   Gross       460 non-null    object 
 9   START_YEAR  9568 non-null   object 
 10  END_YEAR    9568 non-null   object 
 11  DIRECTOR    9568 non-null   object 
 12  ACTORS      9568 non-null   object 
dtypes: float64(3), object(10)
memory usage: 1.3+ MB


In [34]:
movies_df.isna().sum()

MOVIES           0
YEAR           542
GENRE           78
RATING           0
ONE-LINE         0
STARS            0
VOTES            0
RunTime          0
Gross         9108
START_YEAR       0
END_YEAR         0
DIRECTOR         0
ACTORS           0
dtype: int64

In [35]:
movies_df['GENRE'] = movies_df.groupby('ONE-LINE')['GENRE'].transform(lambda x: x.fillna(x.mode()[0] if not x.mode().empty else 'No Specified Genre'))

In [36]:
movies_df['Gross'] = movies_df['Gross'].fillna(0)

In [37]:
movies_df = movies_df.drop(columns=['YEAR', 'STARS'], errors='ignore')


In [38]:
movies_df.head(5)

,MOVIES,GENRE,RATING,ONE-LINE,VOTES,RunTime,Gross,START_YEAR,END_YEAR,DIRECTOR,ACTORS
0,Blood Red Sky,"Action, Horror, Thriller",6.1,A woman with a mysterious illness is forced in...,21062.0,121.00,0,2021.0,2021.0,Peter Thorwarth,"Peri Baumeister, Carl Anton Koch, Alexander Sc..."
1,Masters of the Universe: Revelation,"Animation, Action, Adventure",5.0,The war for Eternia begins again in what may b...,17870.0,25.00,0,2021.0,No Specified Date,Unknown,"Chris Wood, Sarah Michelle Gellar, Lena Headey..."
2,The Walking Dead,"Drama, Horror, Thriller",8.2,Sheriff Deputy Rick Grimes wakes up from a com...,885805.0,44.00,0,2010.0,2022.0,Unknown,"Andrew Lincoln, Norman Reedus, Melissa McBride..."
3,Rick and Morty,"Animation, Adventure, Comedy",9.2,An animated series that follows the exploits o...,414849.0,23.00,0,2013.0,No Specified Date,Unknown,"Justin Roiland, Chris Parnell, Spencer Grammer..."
4,Army of Thieves,"Action, Crime, Horror",0.0,"A prequel, set before the events of Army of th...",0.0,68.87,0,2021.0,2021.0,Matthias Schweighöfer,"Matthias Schweighöfer, Nathalie Emmanuel, Ruby..."


In [39]:
movies_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9568 entries, 0 to 9998
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   MOVIES      9568 non-null   object 
 1   GENRE       9568 non-null   object 
 2   RATING      9568 non-null   float64
 3   ONE-LINE    9568 non-null   object 
 4   VOTES       9568 non-null   float64
 5   RunTime     9568 non-null   float64
 6   Gross       9568 non-null   object 
 7   START_YEAR  9568 non-null   object 
 8   END_YEAR    9568 non-null   object 
 9   DIRECTOR    9568 non-null   object 
 10  ACTORS      9568 non-null   object 
dtypes: float64(3), object(8)
memory usage: 1.1+ MB
